In [10]:
pip install ipykernel ollama pdfplumber pandas yfinance


     |████████████████████████████████| 129 kB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 411 kB 98.9 MB/s eta 0:00:01
     |████████████████████████████████| 427 kB 58.5 MB/s eta 0:00:01
  Using cached websockets-15.0.1-cp39-cp39-macosx_11_0_arm64.whl (173 kB)
     |████████████████████████████████| 3.0 MB 58.7 MB/s eta 0:00:01
Using legacy 'setup.py install' for multitasking, since package 'wheel' is not installed.
    Running setup.py install for multitasking ... done
You should consider upgrading via the '/Users/royvelasquezvalenzuela/Developer/ia/stock/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [25]:
import yfinance as yf
import pandas as pd

ticker = yf.Ticker("AAPL")

# Estados financieros (anuales)
income = ticker.financials            # Estado de resultados
balance = ticker.balance_sheet        # Balance general
cashflow = ticker.cashflow            # Flujo de caja

income, balance, cashflow



/Users/royvelasquezvalenzuela/Developer/ia/stock/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


(                                                      2025-09-30  \
 Tax Effect Of Unusual Items                         0.000000e+00   
 Tax Rate For Calcs                                  1.560000e-01   
 Normalized EBITDA                                   1.447480e+11   
 Net Income From Continuing Operation Net Minori...  1.120100e+11   
 Reconciled Depreciation                             1.169800e+10   
 Reconciled Cost Of Revenue                          2.209600e+11   
 EBITDA                                              1.447480e+11   
 EBIT                                                1.330500e+11   
 Net Interest Income                                          NaN   
 Interest Expense                                             NaN   
 Interest Income                                              NaN   
 Normalized Income                                   1.120100e+11   
 Net Income From Continuing And Discontinued Ope...  1.120100e+11   
 Total Expenses                   

In [26]:
def to_long(df, statement_name):
    out = df.copy()
    out.index.name = "Cuenta"
    out = out.reset_index().melt(id_vars="Cuenta", var_name="Periodo", value_name="Valor")
    out["Estado"] = statement_name
    return out

income_long = to_long(income, "Income Statement")
balance_long = to_long(balance, "Balance Sheet")
cashflow_long = to_long(cashflow, "Cash Flow")

df_all = pd.concat([income_long, balance_long, cashflow_long], ignore_index=True)
df_all.head(10)



,Cuenta,Periodo,Valor,Estado
0,Tax Effect Of Unusual Items,2025-09-30 00:00:00,0.000000e+00,Income Statement
1,Tax Rate For Calcs,2025-09-30 00:00:00,1.560000e-01,Income Statement
2,Normalized EBITDA,2025-09-30 00:00:00,1.447480e+11,Income Statement
3,Net Income From Continuing Operation Net Minor...,2025-09-30 00:00:00,1.120100e+11,Income Statement
4,Reconciled Depreciation,2025-09-30 00:00:00,1.169800e+10,Income Statement
5,Reconciled Cost Of Revenue,2025-09-30 00:00:00,2.209600e+11,Income Statement
6,EBITDA,2025-09-30 00:00:00,1.447480e+11,Income Statement
7,EBIT,2025-09-30 00:00:00,1.330500e+11,Income Statement
8,Net Interest Income,2025-09-30 00:00:00,NaN,Income Statement
9,Interest Expense,2025-09-30 00:00:00,NaN,Income Statement


In [27]:
def to_long(df, statement_name):
    out = df.copy()
    out.index.name = "Cuenta"
    out = out.reset_index().melt(id_vars="Cuenta", var_name="Periodo", value_name="Valor")
    out["Estado"] = statement_name
    return out

income_long = to_long(income, "Income Statement")
balance_long = to_long(balance, "Balance Sheet")
cashflow_long = to_long(cashflow, "Cash Flow")

df_all = pd.concat([income_long, balance_long, cashflow_long], ignore_index=True)
df_all.head(10)



,Cuenta,Periodo,Valor,Estado
0,Tax Effect Of Unusual Items,2025-09-30 00:00:00,0.000000e+00,Income Statement
1,Tax Rate For Calcs,2025-09-30 00:00:00,1.560000e-01,Income Statement
2,Normalized EBITDA,2025-09-30 00:00:00,1.447480e+11,Income Statement
3,Net Income From Continuing Operation Net Minor...,2025-09-30 00:00:00,1.120100e+11,Income Statement
4,Reconciled Depreciation,2025-09-30 00:00:00,1.169800e+10,Income Statement
5,Reconciled Cost Of Revenue,2025-09-30 00:00:00,2.209600e+11,Income Statement
6,EBITDA,2025-09-30 00:00:00,1.447480e+11,Income Statement
7,EBIT,2025-09-30 00:00:00,1.330500e+11,Income Statement
8,Net Interest Income,2025-09-30 00:00:00,NaN,Income Statement
9,Interest Expense,2025-09-30 00:00:00,NaN,Income Statement


In [29]:
def get_value(df, account_contains, period=None):
    # busca la primera fila que contenga el texto
    row = df[df.index.str.contains(account_contains, case=False, na=False)]
    if row.empty:
        return None
    if period is None:
        period = df.columns[0]  # el primer periodo disponible
    return float(row[period].iloc[0])

period = balance.columns[0]

current_assets = get_value(balance, "Total Current Assets", period)
current_liab = get_value(balance, "Total Current Liabilities", period)

if current_assets is None or current_liab is None or current_liab == 0:
    print("No encontrado o no calculable con los campos disponibles.")
else:
    current_ratio = current_assets / current_liab
    print("Periodo:", period)
    print("Liquidez corriente:", current_ratio)


No encontrado o no calculable con los campos disponibles.


In [12]:
import re
import pandas as pd
import pdfplumber
import ollama

MODEL = "llama3.2"   # cambia si usas otro


In [13]:
def extract_pages_text(pdf_path: str):
    pages = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text() or ""
            pages.append({"page": i, "text": text})
    return pages


In [24]:
df = extract_pages_text(pdf_path)
df


Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P2' is an invalid float value
Cannot set gray non-stroke color because /'P3' is an invalid float value
Cannot set gray non-stroke color because /'P4' is an invalid float value
Cannot set gray non-stroke color because /'P5' is an invalid float value
Cannot set gray non-stroke color because /'P6' is an invalid float value
Cannot set gray non-stroke color because /'P7' is an invalid float value
Cannot set gray non-stroke color because /'P8' is an invalid float value
Cannot set gray non-stroke color because /'P9' is an invalid float value
Cannot set gray non-stroke color because /'P10' is an invalid float value
Cannot set gray non-stroke color because /'P11' is an invalid float value
Cannot set gray non-stroke color because /'P12' is an invalid float value
Cannot set gray non-stroke color because /'P13' 

[{'page': 1,
  'text': "K E Y M E T R I C S Q U A R T E R L Y\n(Unaudited)\n0.5\n0.4\n0.3\n0.2\n0.1\n0.0\nVehicle Deliveries Operating Cash Flow ($B) Net Income ($B) (2)\n(millions of units) Free Cash Flow ($B) (1) Adjusted EBITDA ($B) (2) (3)\n(1) Beginning in Q1'25, Capital expenditures is presented inclusive of purchases of solar energy systems and all prior periods have been adjusted.\n(2) As a result of the adoption of the new crypto assets standard, the previously reported quarterly periods in 2024 have been recast.\n21\n(3) Beginning in Q1'25, Adjusted EBITDA (non-GAAP) is presented net of digital assets gains and losses and all prior periods have been adjusted.\n2202-Q3 2202-Q4 3202-Q1 3202-Q2 3202-Q3 3202-Q4 4202-Q1 4202-Q2 4202-Q3 4202-Q4 5202-Q1 5202-Q2\n7\n6\n5\n4\n3\n2\n1\n0\n-1\n-2\n-3\n2202-Q3 2202-Q4 3202-Q1 3202-Q2 3202-Q3 3202-Q4 4202-Q1 4202-Q2 4202-Q3 4202-Q4 5202-Q1 5202-Q2\n8\n7\n6\n5\n4\n3\n2\n1\n0\n2202-Q3 2202-Q4 3202-Q1 3202-Q2 3202-Q3 3202-Q4 4202-Q1 4202-Q2 

In [14]:
def retrieve_pages(pages, question, k=3):
    q = question.lower()
    keywords = re.findall(r"[a-záéíóúñ0-9]+", q)

    scored = []
    for p in pages:
        t = p["text"].lower()
        score = sum(1 for w in keywords if w in t)
        scored.append((score, p))

    scored.sort(key=lambda x: x[0], reverse=True)
    best = [p for s, p in scored if s > 0][:k]
    return best if best else pages[:min(k, len(pages))]


In [19]:
SYSTEM = """
Eres un analista financiero. Responde SOLO usando la evidencia del documento proporcionado.
Reglas:
- No inventes cifras ni conceptos que no estén en el documento. Si falta información, di: "No encontrado en el documento".
- Siempre cita páginas en el formato: (pág. X).
- Si das un cálculo, muestra fórmula y valores usados.
- Si detectas inconsistencias o datos raros, indícalo como "Alerta".
Formato de salida:
1) Respuesta
2) Evidencia (citas)
3) Cálculos (si aplica)
4) Alertas (si aplica)
5) Recomendación breve
6) Ratio de finanzas (si aplica)
"""



In [22]:
def ask_ollama(question, context_pages, model=MODEL):
    context = "\n\n".join([f"[pág. {p['page']}]\n{p['text']}" for p in context_pages])

    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"Pregunta: {question}\n\nDocumento:\n{context}"}
    ]

    resp = ollama.chat(
        model=model,
        messages=messages,
        options={"temperature": 0.2, "top_p": 0.9}
    )
    return resp["message"]["content"]



In [23]:
pdf_path = "/Users/royvelasquezvalenzuela/Developer/ia/stock/TSLA-Q2-.pdf"
pages = extract_pages_text(pdf_path)

question = "Resume los principales hallazgos y riesgos financieros."
ctx = retrieve_pages(pages, question, k=3)
print(ask_ollama(question, ctx))


Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P2' is an invalid float value
Cannot set gray non-stroke color because /'P3' is an invalid float value
Cannot set gray non-stroke color because /'P4' is an invalid float value
Cannot set gray non-stroke color because /'P5' is an invalid float value
Cannot set gray non-stroke color because /'P6' is an invalid float value
Cannot set gray non-stroke color because /'P7' is an invalid float value
Cannot set gray non-stroke color because /'P8' is an invalid float value
Cannot set gray non-stroke color because /'P9' is an invalid float value
Cannot set gray non-stroke color because /'P10' is an invalid float value
Cannot set gray non-stroke color because /'P11' is an invalid float value
Cannot set gray non-stroke color because /'P12' is an invalid float value
Cannot set gray non-stroke color because /'P13' 

**Resumen de hallazgos y riesgos financieros**

El documento proporcionado es un informe financiero detallado que cubre varios aspectos clave del rendimiento y la posición financiera de una empresa. A continuación, se presentan los principales hallazgos y riesgos financieros identificados:

**Hallazgos**

* La empresa ha experimentado un crecimiento en las ventas de vehículos durante el período reportado.
* El flujo de efectivo operativo ha sido positivo en la mayoría de los periodos, con una excepción en Q3-2024.
* El flujo de efectivo de inversión ha sido negativo en algunos periodos, lo que sugiere una alta inversión en activos y una posible disminución en el valor de estos activos.
* La empresa ha reportado una ganancia neto significativa en algunos periodos, lo que indica una rentabilidad sólida.

**Riesgos financieros**

* Alerta: La empresa ha reportado pérdidas significativas en algunos periodos, lo que podría afectar su solidez financiera.
* Alerta: La alta inversión en activo

In [ ]:
def chat_loop(pages, model=MODEL, k=3):
    while True:
        q = input("\nTú: ")
        if q.lower() in ("salir", "exit", "q"):
            break
        ctx = retrieve_pages(pages, q, k=k)
        ans = ask_ollama(q, ctx, model=model)
        print("\nBot:\n", ans)

# chat_loop(pages)


In [1]:
print("Installing Jupyter, Ollama, PDFPlumber, and Pandas...")


Installing Jupyter, Ollama, PDFPlumber, and Pandas...
